# 第五章：格式化输出和为GPT预设响应

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。


In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    # 新增了prefill参数用于预填充文本，默认值为空字符串
    def get_completion(prompt: str, system_prompt="", prefill=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示内容
            system_prompt (str): 系统提示（可选）
            prefill (str): 预填充文本（可选），用于引导GPT的响应开始
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 如果有预填充内容，添加助手消息
        if prefill:
            messages.append({"role": "assistant", "content": prefill})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 使用的模型名称
            messages=messages,             # 消息列表
            max_tokens=2000,              # 最大生成的token数量
            temperature=0.0               # 温度参数，0表示更确定性的回答
        )
        
        # 如果有预填充内容，将其与生成的内容组合
        if prefill:
            return prefill + response.choices[0].message.content
        else:
            return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程

**GPT可以以多种方式格式化其输出**。您只需要明确要求它这样做！

其中一种方式是使用XML标签将响应与其他多余文本分离。您已经学会了可以使用XML标签使您的提示对GPT更清晰、更易解析。事实证明，您还可以要求GPT**使用XML标签使其输出对人类更清晰、更容易理解**。

### 技术要点

1. **XML标签格式化**：指导GPT使用结构化的XML标签来组织输出
2. **预设响应（Prefill）**：通过在assistant角色中预填充部分内容来引导GPT的响应方向
3. **结构化输出**：要求GPT按照特定的格式（如JSON、表格、列表等）输出内容
4. **多段落组织**：使用标记来分离不同类型的信息


### 示例 1：使用XML标签格式化输出

要求GPT使用XML标签来组织其回答，使输出更加结构化和易于解析。


In [2]:
# 示例1：XML标签格式化输出
prompt = """
请分析人工智能对教育行业的影响，并使用以下XML标签格式化你的回答：

<分析>
在这里提供主要分析内容
</分析>

<优势>
列出AI在教育中的优势
</优势>

<挑战>
列出AI在教育中面临的挑战
</挑战>

<建议>
提供实施建议
</建议>
"""

response = get_completion(prompt)
print("🤖 GPT回答:")
print(response)


🤖 GPT回答:
<分析>
人工智能正在改变教育行业的多个方面，包括个性化学习、教学效率和教育资源的获取。AI技术可以通过分析学生的学习行为和成绩数据，提供定制化的学习路径和资源，从而提高学习效果。此外，AI还可以自动化许多教学任务，如评分和课程规划，减轻教师的负担并提高教学效率。
</分析>

<优势>
1. 个性化学习：AI可以根据学生的学习习惯和能力，提供量身定制的学习计划。
2. 提高教学效率：自动化评分和课程规划减少了教师的重复性工作。
3. 扩大教育资源的获取：AI可以帮助开发更丰富的在线学习资源，使更多学生能够获得优质教育。
4. 数据驱动决策：通过分析大量教育数据，AI可以帮助教育机构做出更明智的决策。
</优势>

<挑战>
1. 数据隐私和安全：学生数据的收集和使用可能引发隐私和安全问题。
2. 技术依赖：过度依赖AI可能导致教师角色的弱化和教育质量的下降。
3. 资源不均：在技术资源有限的地区，AI的应用可能加剧教育不平等。
4. 伦理问题：AI在教育中的应用可能引发伦理争议，如算法偏见和公平性问题。
</挑战>

<建议>
1. 加强数据保护：制定严格的数据隐私政策，确保学生信息的安全。
2. 平衡技术与人文：在使用AI的同时，保持教师在教育中的核心作用。
3. 提供技术支持：为资源匮乏地区提供必要的技术支持，确保AI的公平应用。
4. 进行伦理审查：在AI系统开发和应用过程中，进行全面的伦理审查，确保公平和透明。
</建议>


In [3]:
# 示例2：使用预设响应引导GPT风格
prompt = """
作为一名资深的Python开发专家，请解释什么是装饰器，并提供一个实用的示例。
"""

# 使用预设响应来引导GPT以特定的风格和格式回答
prefill = """## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，"""

response = get_completion(prompt, prefill=prefill)
print("🤖 使用预设响应的回答:")
print(response)
print("\n" + "="*60 + "\n")

# 对比：不使用预设响应的回答
response_normal = get_completion(prompt)
print("🤖 普通回答:")
print(response_normal)


🤖 使用预设响应的回答:
## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，装饰器是Python中的一种设计模式，允许开发者在不修改原有函数代码的情况下，动态地增加功能。它本质上是一个函数，接受另一个函数作为参数，并返回一个新的函数。装饰器通常用于日志记录、访问控制、性能计时、缓存等场景。

### 装饰器的基本结构

一个简单的装饰器通常包括以下几个部分：

1. **定义装饰器函数**：接受一个函数作为参数。
2. **定义内部函数**：在内部函数中实现增强功能。
3. **返回内部函数**：将增强后的功能返回。

### 实用示例

下面是一个简单的示例，展示如何使用装饰器来记录函数的执行时间：

```python
import time

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 记录开始时间
        result = func(*args, **kwargs)  # 执行原函数
        end_time = time.time()  # 记录结束时间
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

# 使用装饰器
@timer_decorator
def example_function(n):
    """一个简单的示例函数，计算从0到n的和"""
    total = 0
    for i in range(n):
        total += i
    return total

# 调用函数
result = example_function(1000000)
print(f"Result: {result}")
```

### 解释

- **`timer_decorator`**：这是一个装饰器函数，它接受一个函数`func`作为参数。
- **`wrapper`**：这是内部函数，负责记录执行时间并调用

### 示例 2：JSON格式输出

要求GPT输出标准的JSON格式，便于程序解析和处理。


In [4]:
# 示例3：JSON格式输出
prompt = """
请为一个在线书店设计一个图书数据结构，包含以下信息：
- 书名
- 作者
- ISBN
- 价格
- 分类
- 出版日期
- 库存数量
- 评分

请以标准JSON格式输出，并提供3本示例图书的数据。

输出格式要求：
1. 使用中文字段名
2. 价格用数字类型
3. 评分范围0-5
4. 出版日期格式：YYYY-MM-DD
"""

response = get_completion(prompt)
print("🤖 JSON格式输出:")
print(response)

🤖 JSON格式输出:
```json
[
    {
        "书名": "活着",
        "作者": "余华",
        "ISBN": "9787506365437",
        "价格": 39.00,
        "分类": "文学",
        "出版日期": "2012-08-01",
        "库存数量": 120,
        "评分": 4.8
    },
    {
        "书名": "三体",
        "作者": "刘慈欣",
        "ISBN": "9787536692930",
        "价格": 45.00,
        "分类": "科幻",
        "出版日期": "2008-01-01",
        "库存数量": 200,
        "评分": 4.9
    },
    {
        "书名": "解忧杂货店",
        "作者": "东野圭吾",
        "ISBN": "9787544270878",
        "价格": 38.00,
        "分类": "小说",
        "出版日期": "2014-05-01",
        "库存数量": 150,
        "评分": 4.7
    }
]
```


In [5]:
# 实验1：自定义XML标签格式
# 在这里尝试你自己的格式化提示词

custom_prompt = """
# 请在这里输入你的提示词，尝试使用自定义的XML标签格式
# 例如：分析一个技术趋势、评估一个产品、制定一个计划等
"""

# 取消下面的注释来运行你的实验
# response = get_completion(custom_prompt)
# print("🔬 你的实验结果:")
# print(response)

print("💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码")


💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码


In [6]:
# 实验2：预设响应风格测试
# 选择一个你感兴趣的主题
your_prompt = "请解释区块链技术的工作原理"

# 尝试不同的预设响应风格
prefill_styles = {
    "学术风格": "# 区块链技术原理分析\n\n## 摘要\n\n区块链作为一种分布式账本技术，",
    "科普风格": "👋 大家好！今天我们来聊聊区块链这个神奇的技术！\n\n想象一下，",
    "技术文档风格": "## 区块链技术文档\n\n### 1. 概述\n\n区块链（Blockchain）是",
    "问答风格": "Q: 什么是区块链技术？\nA: 区块链是一种",
}

print("🎨 不同风格的预设响应效果对比：")
print("="*60)

for style_name, prefill in prefill_styles.items():
    print(f"\n📝 {style_name}:")
    print("-" * 30)
    response = get_completion(your_prompt, prefill=prefill)
    # 只显示前200个字符作为预览
    preview = response[:200] + "..." if len(response) > 200 else response
    print(preview)
    print()

# 你也可以尝试自己的预设响应
print("\n🔧 尝试你自己的预设响应:")
your_prefill = """
# 在这里输入你想要的预设响应开头
"""

# 取消注释来测试你的预设响应
# your_response = get_completion(your_prompt, prefill=your_prefill)
# print("🎯 你的预设响应效果:")
# print(your_response)


🎨 不同风格的预设响应效果对比：

📝 学术风格:
------------------------------
# 区块链技术原理分析

## 摘要

区块链作为一种分布式账本技术，区块链技术是一种去中心化的数字记录系统，旨在确保数据的透明性、安全性和不可篡改性。它通过一系列技术和机制来实现这些目标。以下是区块链技术的工作原理的详细解释：

## 1. 数据结构

区块链由一系列“区块”组成，每个区块包含若干交易记录。这些区块通过加密哈希函数链接在一起，形成一个链式结构。每个区块包含以下主要元素：
- **...


📝 科普风格:
------------------------------
👋 大家好！今天我们来聊聊区块链这个神奇的技术！

想象一下，当然！区块链技术可以被视为一个去中心化的分布式账本系统，它通过一系列的区块来记录和存储信息。每个区块都包含若干条交易记录，并与前一个区块通过加密算法相连，形成一个链条，故称为“区块链”。

以下是区块链技术的工作原理的几个关键点：

1. **去中心化**：传统的数据库通常由一个中心化的机构来管理，而区块链是分布式的，数据存储在多个节点...


📝 技术文档风格:
------------------------------
## 区块链技术文档

### 1. 概述

区块链（Blockchain）是### 2. 基本原理

区块链是一种分布式账本技术，允许多个参与者在没有中央权威的情况下共同维护一个数据记录。其核心思想是通过密码学和共识机制来确保数据的安全性和一致性。

### 3. 结构

区块链由一系列区块组成，每个区块包含若干交易记录。区块之间通过加密哈希函数链接，形成一个链式结构。每个区块通常包括以下几个部...


📝 问答风格:
------------------------------
Q: 什么是区块链技术？
A: 区块链是一种区块链是一种去中心化的分布式账本技术，用于安全地记录和验证交易。它由一系列按时间顺序链接的“区块”组成，每个区块包含若干交易记录。以下是区块链技术的基本工作原理：

1. **去中心化**：区块链网络由多个节点组成，每个节点都有一份完整的账本副本。这种去中心化的结构消除了对中央权威的需求，提高了系统的透明度和安全性。

2. **数据结构**：区块链中的...




### 示例 3：复合格式输出

结合多种格式化技术，创建复杂的结构化输出。


In [7]:
# 实验3：复合格式输出
prompt = """
请为一个新的社交媒体应用制定产品规划，要求：

1. 使用XML标签组织内容结构
2. 在技术部分使用表格格式
3. 在时间规划部分使用JSON格式
4. 包含代码示例使用代码块格式

具体要求：
<产品概述>
简要描述产品定位和核心功能
</产品概述>

<目标用户>
用户画像分析
</目标用户>

<功能规划>
核心功能列表和优先级（用表格格式）
</功能规划>

<技术架构>
技术选型对比表格（包含前端、后端、数据库等）
</技术架构>

<开发计划>
6个月的开发时间线（JSON格式，包含阶段、任务、时间等字段）
</开发计划>

<API示例>
提供用户注册接口的代码示例
</API示例>
"""

# 使用预设响应确保格式正确
prefill = """📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的"""

response = get_completion(prompt, prefill=prefill)
print("🔄 复合格式化输出:")
print(response)


🔄 复合格式化输出:
📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的<产品概述>
本产品是一个面向年轻用户的社交媒体应用，旨在提供一个互动性强、内容丰富的平台。核心功能包括实时聊天、动态分享、兴趣小组、以及个性化推荐系统。用户可以通过该平台分享生活点滴、参与话题讨论，并发现与自己兴趣相符的内容和群体。
</产品概述>

<目标用户>
目标用户主要是18-35岁的年轻人，他们对社交互动有较高需求，喜欢分享生活、参与讨论，并希望通过社交平台发现新内容和结识新朋友。这些用户通常是大学生、年轻职场人士以及对新技术敏感的群体。
</目标用户>

<功能规划>
| 功能名称       | 描述                     | 优先级   |
|----------------|--------------------------|----------|
| 实时聊天       | 用户之间可以进行实时聊天 | 高       |
| 动态分享       | 用户可以发布动态和图片   | 高       |
| 兴趣小组       | 用户可以加入或创建小组   | 中       |
| 个性化推荐     | 根据用户兴趣推荐内容     | 中       |
| 用户资料管理   | 用户可以编辑个人资料     | 低       |
| 通知系统       | 提醒用户消息和活动       | 低       |
</功能规划>

<技术架构>
| 技术组件       | 选型对比                 | 选择理由                     |
|----------------|--------------------------|------------------------------|
| 前端框架       | React vs Angular         | React，因其组件化和社区支持 |
| 后端框架       | Node.js vs Django        | Node.js，因其异步处理能力   |
| 数据库         | MongoDB vs PostgreSQL    | MongoDB，因其灵活的文档结构 |
| 云服务      

---

## 📚 总结与最佳实践

### 格式化输出的关键要点：

1. **明确性**：明确告诉GPT你期望的输出格式
2. **一致性**：使用一致的标签和结构来组织信息
3. **可解析性**：确保输出格式便于程序或人类解析
4. **适用性**：根据用途选择合适的格式（JSON、XML、表格等）

### 预设响应的最佳实践：

1. **风格引导**：通过预设响应建立特定的回答风格
2. **格式控制**：确保输出格式的一致性
3. **专业角色**：让GPT扮演特定的专业角色
4. **长度控制**：预设响应应该简洁但足够引导方向

### 实际应用场景：

- 📊 **数据分析报告**：使用XML标签组织分析结果
- 🔧 **技术文档**：结合代码块和表格展示技术方案
- 📋 **项目计划**：使用JSON格式输出结构化的项目信息
- 🎯 **产品评估**：通过预设响应确保评估的专业性和一致性

通过掌握这些格式化技术，你可以让GPT输出更加结构化、专业化的内容，大大提升工作效率！
